In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, precision_recall_curve
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold,StratifiedKFold

from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

from scipy.stats import pearsonr, chi2_contingency
from itertools import combinations
from statsmodels.stats.proportion import proportion_confint

import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_white'

### Data Preprocessing

In [3]:
# import data
data = pd.read_csv(
    'accepted_2007_to_2018Q4.csv',
    usecols=[2, 5, 6, 8, 11, 12, 13, 15, 16],
    parse_dates = ['issue_d'], 
    infer_datetime_format = True
)
data = data.reset_index(drop=True)

In [4]:
# Features
features = [x for x in data.columns if x not in ['loan_status']]
X = data[features]

In [5]:
# Target
y = data['loan_status'].copy()
y = (~y.isin(['Current', 'Fully Paid', 'In Grace Period'])).astype('int')

In [6]:
# fill missing values
Missing = ['loan_amnt','int_rate','annual_inc']
X[Missing] = X[Missing].fillna(X[Missing].mean())

D:\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
# set 'home_ownership'
# Only addr_state, grade and sec_app_earliest_cr_line are remained
for cat_feat in ['grade', 'term', 'emp_length', 'home_ownership']:
    dummies_df = pd.get_dummies(X[cat_feat]).rename(columns=lambda x: cat_feat + str(x))
    X = pd.concat([X, dummies_df], axis=1)
    X.drop(cat_feat,axis=1, inplace=True)

In [8]:
X_trainval = X[(data.issue_d < '2018-01-01 00:00:00')]
X_test = X[(data.issue_d >= '2018-01-01 00:00:00')]
y_trainval = y[(data.issue_d < '2018-01-01 00:00:00')]
y_test = y[(data.issue_d >= '2018-01-01 00:00:00')]

In [9]:
print(f'Train set has {X_trainval.shape[0]} rows while test set has {X_test.shape[0]} rows')

Train set has 1765426 rows while test set has 495242 rows


In [10]:
X_trainval.drop('issue_d',axis=1, inplace=True)
X_test.drop('issue_d',axis=1, inplace=True)

D:\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Default Risk Prediction Model

In [14]:
params_lgb = {'num_leaves': 20,
          'min_data_in_leaf': 10,
          'objective': 'binary',
          'max_depth': 5,
          'learning_rate': 0.01,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'logloss',
          "verbosity": 0
          }

In [ ]:
NFOLDS = 5
folds = KFold(n_splits=NFOLDS)

columns = X_trainval.columns
splits = folds.split(X_trainval, y_trainval)
y_preds_lgb = np.zeros(X_test.shape[0])
y_oof_lgb = np.zeros(X_trainval.shape[0])
  
for fold_n, (train_index, valid_index) in enumerate(splits):
    #print('Fold:',fold_n+1)
    X_train, X_valid = X_trainval[columns].iloc[train_index], X_trainval[columns].iloc[valid_index]
    y_train, y_valid = y_trainval.iloc[train_index], y_trainval.iloc[valid_index]
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid)

    clf = lgb.train(params_lgb, dtrain, 10000, valid_sets = [dtrain, dvalid], verbose_eval=200)
    
    y_pred_valid = clf.predict(X_valid)
    y_oof_lgb[valid_index] = y_pred_valid
    
    y_preds_lgb += clf.predict(X_test) / NFOLDS
    
    del X_train, X_valid, y_train, y_valid

In [11]:
y_preds_lgb = pd.read_csv('y_pred.csv',index_col = 0, header = None)

In [ ]:
y_preds_lgb.columns = ['Default Probability']
fig = px.histogram(y_preds_lgb, x='Default Probability')
fig.update_layout(xaxis_title='Default Probability', yaxis_title='Number of loans')

# Optimization

In [ ]:
x_data = data[features]
df_test = x_data[(data.issue_d >= '2018-01-01 00:00:00')]
df_test['D'] = y_preds_lgb

In [14]:
# select the Mar of 2018
mon_df = df_test[df_test['issue_d'].isin(['2018-03-01'])]

In [15]:
sample = mon_df.sample(n=100, replace=False, random_state=11)
sample['term'] = re.findall('([0-9]+?)\smonths',str(sample['term'].values))
k = np.zeros(100)
for i in range(100):
    k[i] = sample.iloc[i,0]*(1+sample.iloc[i,2]*float(sample.iloc[i,1])/1200)

In [16]:
import pyomo.environ as pe

# create a model
model = pe.ConcreteModel()

# define decision variables
n = 100
# B is the total amount
B = 500000
# R is the risk 
R = 0.02
# whether to lend 
model.x = pe.Var(range(n), domain = pe.Binary)

# declare objective
model.re = pe.Objective(expr = sum(model.x[i]*k[i] for i in range(n)), sense = pe.maximize)

In [18]:
# constraint: less than total amount
model.amt = pe.Constraint(expr = sum(sample.iloc[i,0]*model.x[i] for i in range(n)) <= B)

# constraint: risk limit
model.risk = pe.Constraint(expr = sum(sample.iloc[i,0]*sample.iloc[i,8]*model.x[i] for i in range(n))/B <= R)

# constraint: money liquidity
model.liquidity = pe.Constraint(
    expr = sum(int(sample.iloc[i,1]) * sample.iloc[i,0] * model.x[i] for i in range(n)) <= B*(36*0.4+60*0.6)
)

# constraint: upper limit of number of selected loans
model.total = pe.Constraint(
    expr = sum(model.x[i] for i in range(n)) <= 50
)


solver = pe.SolverFactory('gurobi')
results = solver.solve(model)

In [19]:
print(model.re()/B, model.risk())

1.701173165 0.01998685340873012


In [49]:
lst_x = np.zeros(n)
lst_loan = pd.Series(n)
for i in range(n):
    lst_x[i] = int(model.x[i]())
    lst_loan[i] = 'loan ' + str(i+1)

In [50]:
loans = pd.concat([lst_loan,pd.Series(lst_x),], axis = 1)
loans.columns = ['loans', 'choose_or_not']
loans

,loans,choose_or_not
0,loan 1,0.0
1,loan 2,1.0
2,loan 3,0.0
3,loan 4,0.0
4,loan 5,0.0
...,...,...
95,loan 96,1.0
96,loan 97,1.0
97,loan 98,0.0
98,loan 99,0.0


In [ ]:
fig = px.bar(loans[0:50], x = 'loans', y = 'choose_or_not', text='loans',height=300)
fig.update_layout()

In [61]:
fig = px.bar(loans[50:], x = 'loans', y = 'choose_or_not', text='loans',height=300)
fig.update_layout()

In [21]:
T = 100 # simulation times
N = 100 # number of borrowers
rr = dict() #return rate
tr = dict() #ture risk

for t in range(T):
    # randomly choose n borrowers of one month
    sample = mon_df.sample(n=N, replace=False, random_state=t)
    sample['term'] = re.findall('([0-9]+?)\s+months',str(sample['term'].values))
    # compute the return amount of each lending project
    k = np.zeros(N)
    for i in range(N):
        k[i] = sample.iloc[i,0]*(1+sample.iloc[i,2]*float(sample.iloc[i,1])/1200)
    import pyomo.environ as pe
    
    R = np.linspace(0.01,0.1,50)
    for j in range(len(R)):
        # create a model
        model = pe.ConcreteModel()

        n = N
        # B is the total amount
        B = 500000
        # whether to lend 
        model.x = pe.Var(range(n), domain = pe.Binary)

        # declare objective
        model.re = pe.Objective(expr = sum(model.x[i]*k[i] for i in range(n)), sense = pe.maximize)

        # constraint: less than total amount
        model.amt = pe.Constraint(expr = sum(sample.iloc[i,0]*model.x[i] for i in range(n)) <= B)

        # constraint: risk limit
        model.risk = pe.Constraint(expr = sum(sample.iloc[i,0]*sample.iloc[i,8]*model.x[i] for i in range(n))/B <= R[j])

        # constraint: money liquidity
        model.liquidity = pe.Constraint(
            expr = sum(int(sample.iloc[i,1]) * sample.iloc[i,0] * model.x[i] for i in range(n)) <= B*(36*0.4+60*0.6)
        )

        # constraint: upper limit of number of selected loans
        model.total = pe.Constraint(
            expr = sum(model.x[i] for i in range(n)) <= 50
        )
        solver = pe.SolverFactory('gurobi')
        results = solver.solve(model)
        rr[t,j] = model.re()/B
        tr[t,j] = model.risk()

In [22]:
def get_stats(R, rr):
    RR = dict()
    for j,risk in enumerate(R):
        RR.update({risk: [rr[t,j] for t in range(T)]})
        ERR = pd.DataFrame(RR).mean()
        stats_ERR = pd.DataFrame(ERR)
        stats_ERR.reset_index(inplace=True)
    return stats_ERR

In [23]:
stats_ERR = get_stats(R, rr)
stats_ERR.columns = ['Risk Rate','Return Rate']
stats_ETR = get_stats(R, tr)
stats_ETR.columns = ['Risk Rate','True Risk']

In [ ]:
fig_ETR = px.scatter(stats_ETR, x='Risk Rate', y='True Risk',color = 'Risk Rate')
fig_ETR.update_layout(showlegend=False)

In [25]:
fig_ERR = px.scatter(stats_ERR, x='Risk Rate', y='Return Rate',color = 'Risk Rate')
fig_ERR.update_layout(showlegend=False)